In [1]:
from pymongo import MongoClient
import pandas as pd

In [2]:
client = MongoClient("localhost:27017")
db = client["ironhack"]
c = db.get_collection("companies")

In [3]:
def mongo_extract (filter):
    projection = {"_id":0, "name":1, "category_code":1, "tag_list":1, "total_money_raised":1, "offices.city":1, "offices.state_code": 1, "offices.country_code":1}
    
    list_ = list(c.find(filter, projection))

    df = pd.DataFrame(list_).explode("offices").reset_index(drop=True)
    df = pd.concat([df, df["offices"].apply(pd.Series)], axis=1).reset_index(drop=True)
    df = df.drop(["offices", 0], axis = 1)
    df = df[df["city"]!=""]
    return df

In [4]:
money_filter = {"total_money_raised" : {"$regex" : "[$€].*[MB]"}}
money_df = mongo_extract(money_filter)
money_df["city"].value_counts()[:5]

San Francisco    436
New York         335
London           173
Seattle          110
Mountain View    106
Name: city, dtype: int64

In [5]:
design_filter = {"tag_list" : {"$regex" : "design"}}
design_df = mongo_extract(design_filter)
design_df["city"].value_counts()[:5]

London           24
New York         23
San Francisco    20
San Diego        11
Chennai          10
Name: city, dtype: int64

In [6]:
gaming_filter = {"category_code" : "games_video"}
gaming_df = mongo_extract(gaming_filter)
gaming_df["city"].value_counts()[:5]


New York         75
San Francisco    68
London           36
Los Angeles      31
Paris            17
Name: city, dtype: int64